In [196]:
from Distillation_disc_cont import Simulator
from memory import Memory
from Hybrid_paper import Agent
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
import dill

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [197]:
#dill.load_session('Agent.db')

In [198]:
%autoreload

## Make agent and define hyperparameters

In [199]:
alpha = 1e-3
beta = 1e-4 
gamma = 0.97

num_episodes = int(1e4)
memory_size = int(1e5)
batch_size = 20
pretrain_eps = batch_size

In [200]:
env = Simulator()
memory = Memory(max_size = memory_size)
agent = Agent(env = env, alpha = alpha, beta = beta, gamma = gamma, layer_size = 30)
agent.actor_DQN.summary()
#agent.actor_param.summary()

Model: "model_40"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_state_DQN (InputLayer)    (None, 6)            0                                            
__________________________________________________________________________________________________
input_param_DQN (InputLayer)    (None, 1)            0                                            
__________________________________________________________________________________________________
dense1_state_DQN (Dense)        (None, 30)           210         Input_state_DQN[0][0]            
__________________________________________________________________________________________________
dense1_param_DQN (Dense)        (None, 30)           60          input_param_DQN[0][0]            
___________________________________________________________________________________________

D:\Github\RL-Process-Design\Discrete and Continuous\Hybrid_paper.py:57: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("In..., outputs=Tensor("po...)`
  actor_param = Model(input = input_state, output = policy_continuous)
D:\Github\RL-Process-Design\Discrete and Continuous\Hybrid_paper.py:60: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  actor_DQN = Model(input = [input_state_DQN, input_param_DQN], output = [policy_discrete_probs])


## Fill the experience memory

In [201]:
state = env.reset()

for ep in range(pretrain_eps):
    action = env.discrete_action_space.sample(), env.continuous_action_space.sample()
    next_state, reward, done, _ = env.step(action)
    
    if done:
        next_state = np.zeros(env.observation_space.shape)
        memory.add((state, action, reward, next_state))
        
    else:
        memory.add((state, action, reward, next_state))
        state = next_state
    

In [202]:
score_history = []
for i in range(num_episodes):
    
    done = False
    
    state = env.reset()
    score = 0
    
    while not done:
        action = agent.choose_action_epsgreedy(state, i, num_episodes)
        next_state, reward, done, info = env.step(action)
        #agent.learn(state,  action, reward, next_state, done)
        score += reward
        memory.add((state, action, reward, next_state))
        state = next_state
    
    score_history.append(score)
    next_state = np.zeros(state.shape)
    memory.add((state, action, reward, next_state))
    
    avg_score = np.mean(score_history[-100:]) #average of last 100 scores
    if i%100 == 0:
        print(f'episode {i}, avg_score {avg_score}, last action {action}')
        
    batch = memory.sample(batch_size)
    states = np.array([each[0] for each in batch])
    actions = np.array([each[1] for each in batch])
    rewards = np.array([each[2] for each in batch])
    next_states = np.array([each[3] for each in batch])
    
    agent.learn(states,  actions, rewards, next_states, done, batch_size, verbose = 1)    


episode 0, avg_score -1385.3444752751188, last action (0, array([[-0.47835606]]))


ValueError: An operation has `None` for gradient. Please make sure that all of your ops have a gradient defined (i.e. are differentiable). Common ops without gradient: K.argmax, K.round, K.eval.

In [ ]:
# Check which layer is without gradient?

In [ ]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N

In [203]:
episodes = np.arange(num_episodes)
smoothed_rews = running_mean(score_history, 10)
plt.plot(episodes[-len(smoothed_rews):], smoothed_rews)
plt.plot(episodes, score_history,color='grey', alpha=0.3)
plt.xlabel("steps")
plt.ylabel("reward")
plt.legend(["avg reward", "reward"])

NameError: name 'running_mean' is not defined

In [230]:
#test agent
state = env.reset()
done = False
while not done:
    state = state[np.newaxis, :]
    action_discrete, action_continuous = agent.policy.predict(state)
    action_discrete = np.argmax(action_discrete)
    action = action_discrete, action_continuous
    
    state, reward, done, info = env.step(action)
    print(f'action LK: {action_discrete}, action split: {action_continuous[0]}, reward: {reward}')

action LK: 2, action split: [0.5], reward: -0.12775379014095772
action LK: 2, action split: [0.5], reward: -100
action LK: 2, action split: [0.5], reward: -100
action LK: 2, action split: [0.5], reward: -100
action LK: 2, action split: [0.5], reward: -100
action LK: 2, action split: [0.5], reward: -100
action LK: 2, action split: [0.5], reward: -100
action LK: 2, action split: [0.5], reward: -100
action LK: 2, action split: [0.5], reward: -100
action LK: 2, action split: [0.5], reward: -100
action LK: 2, action split: [0.5], reward: -100


In [231]:
env.stream_table

[array([9.1, 6.8, 9.1, 6.8, 6.8, 6.8]),
 array([9.1       , 6.8       , 4.55000019, 3.4000001 , 0.        ,
        0.        ]),
 array([0.        , 0.        , 4.54999981, 3.3999999 , 6.8       ,
        6.8       ]),
 array([9.1       , 6.8       , 2.2750001 , 1.70000005, 0.        ,
        0.        ]),
 array([0.        , 0.        , 2.2750001 , 1.70000005, 0.        ,
        0.        ]),
 array([0.        , 0.        , 2.27499986, 1.69999993, 0.        ,
        0.        ]),
 array([0.        , 0.        , 2.27499995, 1.69999998, 6.8       ,
        6.8       ]),
 array([9.1       , 6.8       , 1.13750005, 0.85000002, 0.        ,
        0.        ]),
 array([0.        , 0.        , 1.13750005, 0.85000002, 0.        ,
        0.        ]),
 array([0.        , 0.        , 1.13750005, 0.85000002, 0.        ,
        0.        ]),
 array([0.        , 0.        , 1.13750005, 0.85000002, 0.        ,
        0.        ]),
 array([0.        , 0.        , 1.13749993, 0.84999996, 0.  

In [193]:
action_discrete, action_continuous = agent.policy.predict(state[np.newaxis, :])

In [ ]:
#dill.dump_session('Agent.db')